Importação de bibliotecas e Modelo Base, no caso estou usando o modelo VGG16 que peguei no keras: https://keras.io/api/applications/vgg/#vgg16-function

In [15]:
from tensorflow.keras.applications import VGG16

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in base_model.layers:
    layer.trainable = False

Montando o Google Drive, pois estou usando diretamente da pasta PetImages que contem 2 pastas dentro, as pastas Cat e Dog. Pois assim não preciso baixar o arquivo no meu PC e nem colar diretamente no drive pois o processo demoraria muito.

In [16]:
from google.colab import drive
import os
from tensorflow.keras.preprocessing import image
import numpy as np
from tensorflow.keras.applications.vgg16 import preprocess_input
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Adicionando e configurando camadas personalizadas

In [20]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(2, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Preparando os Dados

In [21]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(preprocessing_function=preprocess_input, validation_split=0.2)

train_generator = datagen.flow_from_directory(
    '/content/drive/My Drive/PetImages',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training')

validation_generator = datagen.flow_from_directory(
    '/content/drive/My Drive/PetImages',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation')

Found 584 images belonging to 2 classes.
Found 146 images belonging to 2 classes.


Treinamento do modelo com o dataset de Cat e Dog.

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=10)

Epoch 1/10
10/18 [===============>..............] - ETA: 2:32 - loss: 0.2828 - accuracy: 0.9155

Fazer Predições e processar e exibir resultados de Predições.

In [ ]:
predictions = model.predict(x)

predicted_class_indices = np.argmax(predictions, axis=1)
labels = (train_generator.class_indices)
labels = dict((v, k) for k, v in labels.items())
predictions_labels = [labels[k] for k in predicted_class_indices]
print(predictions_labels)